In [1]:
import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import os
from pathlib import Path
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 80
from matplotlib import rcParams
rcParams['figure.figsize'] = 8,4
sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'

bstar=20
shrink=1

In [2]:
#this function constructs the kernel (not normalized)
#it will be normalized later anyways
@nb.jit(nopython=True)
def kernel(band):
    return np.array([np.power(1-np.power((band-x)/band,2),3) for x in range(1,band+1)]
                    +[np.power(1-np.power(x/band,2),3) for x in range(1,band)])

#naive kernel smoothing implementation
@nb.jit(nopython=True)
def naivepdf(grid, band, pdf, ker):
    imax=len(grid)
    for i in range(0,imax):
        left=max(i-band,0)
        right=min(i+band,imax)
        for j in range(left,right):
            pdf[i]+=grid[j]*ker[j-i+band]
            
#cdf computation
@nb.jit(nopython=True)
def naivecdf(pdf,cdf):
    for i in range(0,len(cdf)):
        for j in range(0,i):
            cdf[i]=cdf[i]+pdf[j]

#this function counts the occurences of each observation and places them on a grid
@nb.jit(nopython=True)
def gcounts(data, grid):
    for x in data:
        grid[x]+=1
        
#this function measures the profit
@nb.jit(nopython=True)
def profits(prof1, prof2, ncdf):
    prof2=np.multiply(prof2, ncdf)
    for i in range(0,len(prof1)):
        prof1[i]=np.argmax(prof2[i,:])
    return prof1

#hazard rate computation
@nb.jit(nopython=True)
def hazard(cdf, pdf):
    return cdf/pdf

#this is the main function
@nb.jit(nopython=True)
def step1(mydata, fc, grid, pdf, cdf, prof2, prof1):
    #pick the bandwidth with Silverman rule
    band=max(int(1.06*2.987*np.power(len(mydata),-1/5)*np.std(mydata)),1)
    ker=kernel(band+1)
    gcounts(mydata, grid)
    naivepdf(grid, band, pdf, ker)
    naivecdf(pdf, cdf)
    ncdf=cdf/np.max(cdf)
    
    #to normalize the density, I need to divide it by the sum
    #but then also rescale it back to [0,100], thus multiply by fc  
    npdf=fc*pdf/np.sum(pdf)
    
    sigma=profits(prof1, prof2, ncdf)
    
    #the inverse hazard rate maps into [0,100]
    haz=hazard(ncdf, npdf)
    return ncdf, npdf, sigma, haz, band

In [3]:
#adaptive kernel smoothing implementation
@nb.jit(nopython=True)
def adaptpdf(grid, band, pdf, lambd):
    imax=len(grid)
    for i in range(0,imax):
        area=0
        myband=min(max(int(band*lambd[i]),1),100*fc)
        #for the adaptive kernel I need to call the kernel() function inside
        #this should be done in matrix form but I am too lazy
        myker=kernel(myband+1)
        left=max(i-myband,0)
        right=min(i+myband,imax)
        for j in range(left,right+1):
            pdf[i]+=float(grid[j]*myker[j-i+myband])
            area+=myker[j-i+myband]
        #reweight the kernel proporionally to the accumulated area
        pdf[i]=float(pdf[i])/float(area)

@nb.jit(nopython=True)
def step2(mydata, grid, adapdf, lambd):
    gcounts(mydata, grid)
    band=max(int(1.06*2.987*np.power(len(mydata),-1/5)*np.std(mydata))/shrink,1)
    
    for i in range(0,bstar*fc):
        lambd[i]=1-(1-bstar*fc/band)*(bstar*fc-i)*(bstar*fc-i)/((bstar*fc)*(bstar*fc))
    for i in range(0,bstar*fc):
        lambd[100*fc-i]=1-(1-bstar*fc/band)*(bstar*fc-i)*(bstar*fc-i)/((bstar*fc)*(bstar*fc))
    
    adaptpdf(grid, band, adapdf, lambd)
    adapdf=fc*adapdf/(np.sum(adapdf))
    return adapdf, band

In [4]:
for fc in [8]:
    print('fc=',fc)
    for industry in ["leg", "mil", "sci", "edu", "tec"]:
        for weight in ['55','80']:
            print(industry, weight, '...')
            [PART,DATA]=pkl.load(open(datadir+"/derive/"+industry+weight+"j.pkl", "rb"))

            mydir=datadir+"/derive/"+industry+weight+"/"

            try:
                os.stat(mydir)
            except:
                os.mkdir(mydir) 

            ##################
            ##################
            
            DATA_res=np.multiply(fc,DATA.sapp_res.values).astype(int)
            mydata=DATA_res

            grid=np.zeros(100*fc+1, int)

            #leave a 1 in the pdf for division
            pdf=np.ones(100*fc+1, float)*0.0001

            cdf=np.zeros(100*fc+1, float)
            prof1=np.zeros(100*fc+1, int)
            prof2=np.array([[th-s for s in range(0,100*fc+1)] for th in range(0,100*fc+1)], int)

            ncdf, npdf, sigma, haz, band = step1(mydata, fc, grid, pdf, cdf, prof2, prof1)

            #addr=mydir+industry+weight+"j_cdfpdf_"+str(fc)+".pkl"
            #pkl.dump([ncdf,npdf],open(addr, "wb"))
            
            haz=np.array(haz)
            addr=mydir+industry+weight+"j_strategy_"+str(fc)+".pkl"
            pkl.dump([sigma,haz],open(addr, "wb"))
            
            ##################
            ##################
            
            DATA_s=np.multiply(fc,DATA.sapp_s.values).astype(int)
            mydata=DATA_s

            grid=np.zeros(100*fc+1, int)

            #leave a 1 in the pdf for division
            pdf=np.ones(100*fc+1, float)*0.0001

            cdf=np.zeros(100*fc+1, float)
            prof12=np.zeros(100*fc+1, int)
            prof22=np.array([[th-s for s in range(0,100*fc+1)] for th in range(0,100*fc+1)], int)

            ncdf2, npdf2, sigma2, haz2, band2 = step1(mydata, fc, grid, pdf, cdf, prof22, prof12)

            haz2=np.array(haz2)
            addr=mydir+industry+weight+"j_strategy2_"+str(fc)+".pkl"
            pkl.dump([sigma2,haz2],open(addr, "wb"))
            
            #addr=mydir+industry+weight+"j_cdfpdf2_"+str(fc)+".pkl"
            #pkl.dump([ncdf2,npdf2],open(addr, "wb"))
            
            ###################
            ###################
            
            frequencies=np.array(PART[1])/np.sum(PART[1])
            powers=np.array(PART[0])
            ncdf3=np.zeros(100*fc+1, float)
            for i in range(0,100*fc+1):
                for j in range(0,len(powers)):
                    ncdf3[i]+=np.power(ncdf2[i],powers[j]-1)*frequencies[j]

            npdf3=np.zeros(100*fc+1, float)  
            for i in range(0,100*fc):
                npdf3[i]=(ncdf3[i+1]-ncdf3[i])*fc

            prof13=np.zeros(100*fc+1, int)
            prof23=np.array([[th-s for s in range(0,100*fc+1)] for th in range(0,100*fc+1)], int)    

            sigma3=profits(prof13, prof23, ncdf3)
            haz3=hazard(ncdf3, npdf3)
            
            addr=mydir+industry+weight+"j_strategy3_"+str(fc)+".pkl"
            pkl.dump([sigma3,haz3],open(addr, "wb"))

fc= 8
leg 55 ...
leg 80 ...
mil 55 ...
mil 80 ...
sci 55 ...
sci 80 ...
edu 55 ...
edu 80 ...
tec 55 ...
tec 80 ...
